Since [OpenPaths](https://openpaths.cc/) is dead, we switched to [WHIB](http://www.bleatinc.com/), which isn't [owned by Facebook](https://moves-app.com/) and doesn't [kill the phones battery](http://www.bigpaua.com/arc/).
A 2$ `premium` feature let's you export all the data as CSV, which is easy to parse and display.

In [1]:
import pandas
import matplotlib.pylab as plt
import seaborn
import mplleaflet
import folium

In [2]:
plt.rcParams['figure.figsize'] = (16, 9)  # We live in a widescreen world

In [3]:
locations = pandas.read_csv('journey.csv')

In [4]:
locations.head()

,Crumb,LocalDate,LocalTime,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
0,0,01.07.17,11:05,46.935376,7.417898,559,20
1,1,01.07.17,11:34,46.939073,7.437170,582,200
2,2,01.07.17,11:34,46.939392,7.437318,526,10
3,3,01.07.17,11:44,46.944279,7.471384,548,1414
4,4,01.07.17,11:44,46.942943,7.474213,567,30


In [5]:
locations.describe()

,Crumb,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
count,716.000000,716.000000,716.000000,716.000000,716.000000
mean,357.500000,46.898296,7.456958,607.250000,383.952514
std,206.835684,0.131176,0.075443,283.111993,1513.610282
min,0.000000,46.434918,7.386615,-43.000000,0.000000
25%,178.750000,46.935388,7.417901,547.000000,48.000000
50%,357.500000,46.937739,7.428722,557.000000,65.000000
75%,536.250000,46.948093,7.444699,559.000000,79.000000
max,715.000000,46.990264,7.711900,2003.000000,23964.000000


In [6]:
# Plot a subset of the data points
# We use a subset of the data to not overwhelm the map...
subset=10
plt.scatter(locations['Longitude'][::subset], locations['Latitude'][::subset])
mplleaflet.display()

In [7]:
# Use folium
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()], tiles='Stamen Toner')
for index, row in locations.iterrows():
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                        radius=10, color='black',
                        fill_color='#3186cc').add_to(m)
m

In [8]:
# Massage data to get a heatmap
from folium.plugins import HeatMap
xy = [[row['Latitude'], row['Longitude']] for index, row in locations.iterrows()]
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()], tiles='Stamen Toner', zoom_start=9)
folium.plugins.HeatMap(xy).add_to(m)
m